In [1]:
import numpy as np
import cv2
import imutils
import sys
import pytesseract
import pandas as pd
import time

In [2]:
# pip install pytesseract
# https://github.com/madmaze/pytesseract
# download from https://github.com/UB-Mannheim/tesseract/wiki and install
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\abhinav.jhanwar\AppData\Local\Tesseract-OCR\tesseract.exe"

In [113]:
image_path = 'test1.jpg'

# read image
image = cv2.imread(image_path)

# resize image
image = imutils.resize(image, width=500)

# show original image
cv2.imshow("Original Image", image)
cv2.waitKey(0)

-1

In [114]:
# 1: convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#cv2.imshow("1 - Grayscale Conversion", gray)

# 2: Apply Bilateral Filter
gray = cv2.bilateralFilter(gray, 11, 20, 20)
#cv2.imshow("2 - Bilateral Filter", gray)

# 3: Get Edges
edged = cv2.Canny(gray, 170, 200)
#cv2.imshow("3 - Canny Edges", edged)

cv2.waitKey(0)
cv2.destroyAllWindows()

# 4: Get all contours 
(cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
# sort countours based on area
cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30]
# get total number of number plates in the image
NumberPlateCnt = None 
count = 0
test_image = image.copy()
samples = []
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4:  
        NumberPlateCnt = approx
        break
        
# 5: take thresh of image to separate text from background
thresh = cv2.threshold(gray, 125, 255, cv2.THRESH_BINARY)[1]
cv2.imshow("Thresh", thresh)
cv2.waitKey(0)

# 6: get proper boundary box
rect = cv2.minAreaRect(NumberPlateCnt)
box = cv2.boxPoints(rect)
box = np.int0(box)

In [115]:
# check the contours    
test_image = image.copy()
cv2.drawContours(test_image, cnts, -1, (0, 0, 255), 5)

# display original image with contours
cv2.imshow("output", test_image)
cv2.waitKey(0)

# display original image with selected contour
test_image = image.copy()
cv2.drawContours(test_image, [box], -1, (0,255,0), 3)
cv2.imshow("checker again",test_image)
cv2.waitKey(0)

-1

**psm: page segmentation modes:**
*  0    Orientation and script detection (OSD) only.
*  1    Automatic page segmentation with OSD.
*  2    Automatic page segmentation, but no OSD, or OCR.
*  3    Fully automatic page segmentation, but no OSD. (Default)
*  4    Assume a single column of text of variable sizes.
*  5    Assume a single uniform block of vertically aligned text.
*  6    Assume a single uniform block of text.
*  7    Treat the image as a single text line.
*  8    Treat the image as a single word.
*  9    Treat the image as a single word in a circle.
* 10    Treat the image as a single character.
* 11    Sparse text. Find as much text as possible in no particular order.
* 12    Sparse text with OSD.
* 13    Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific.


In [116]:
# 6: Masking the part other than the number plate
# create black image as the size of image
mask = np.zeros(thresh.shape, np.uint8)
# create white mask in the selected contour
mask = cv2.drawContours(mask, [box], 0, 255, -1)
new_image = cv2.bitwise_and(thresh, thresh, mask=mask)

# Configuration for tesseract        
config = ('-l eng --oem 3 --psm 3')

# 6: Run tesseract OCR on image
text = pytesseract.image_to_string(new_image, config=config)
print("Text in image:", text)

cv2.imshow("Masked image", new_image)
cv2.waitKey(0)


Text in image: 15-LK-10898


-1